In [ ]:
from mainnet_launch.pages.gas_costs.gas_costs import fetch_tokemak_address_constants_dfs
from mainnet_launch.database.schema.postgres_operations import TableSelector, merge_tables_as_df
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.constants import ETH_CHAIN, ChainData

from mainnet_launch.database.schema.full import Transactions, Blocks, ChainlinkGasCosts
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

In [ ]:
from mainnet_launch.pages.gas_costs.gas_costs import fetch_our_gas_costs_df

df, address_constants = fetch_our_gas_costs_df()

In [ ]:
break

In [ ]:
import streamlit as st

for col in ["category", "label"]:
    st.subheader(f"Gas Costs in ETH and Gas Used by Expense {col}")
    for value_col in ["gas_cost_in_eth", "gas_used"]:
        daily_sum_df = (
            df.groupby([col, "datetime"])[value_col]
            .sum()
            .reset_index()
            .pivot(index="datetime", columns=col, values=value_col)
            .fillna(0)
            .resample("1D")
            .sum()
        )

        st.subheader(f"{value_col} by {col}")
        for n_days in [1, 7, 30]:
            fig = px.bar(
                daily_sum_df.rolling(window=n_days).sum(),
                title=f"{n_days}-day rolling sum Mainnet {value_col}",
            )
            fig.update_yaxes(title_text=value_col)
            st.plotly_chart(fig, use_container_width=True)

In [6]:
# Ensure datetime column is proper and extract the date
df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Create weighted value column
df["gas_price_times_gas_used"] = df["effective_gas_price"] * df["gas_used"] / 1e9

# Group and compute sum of weights and weighted values
grouped = df.groupby(["date", "category"]).agg(
    weighted_sum=("gas_price_times_gas_used", "sum"), total_weight=("gas_used", "sum")
)

# Compute weighted average
grouped["weighted_avg_effective_gas_price"] = grouped["weighted_sum"] / grouped["total_weight"]

# Optionally drop intermediate columns
grouped = (
    grouped.drop(columns=["weighted_sum", "total_weight"])
    .reset_index()
    .pivot(index="date", columns="category", values="weighted_avg_effective_gas_price")
)

px.line(
    grouped,
    title="Weighted Average Effective Gas Price by Date and EOA Address",
    labels={"value": "Weighted Average Effective Gas Price", "date": "Date"},
).update_yaxes(title_text="Effective Gas Price (Gwei)").show()

In [14]:
import pandas as pd
import plotly.express as px

N = 30  # <-- your rolling window size in days

# 1. Parse and index
# df["datetime"] = pd.to_datetime(df["datetime"])
# df = df.set_index("datetime").sort_index()

# 2. Compute per-transaction contributions
df["weighted_contrib"] = df["effective_gas_price"] * df["gas_used"] / 1e9
df["weight"] = df["gas_used"]

# 3. Aggregate to daily sums per category
#    This gives you a MultiIndex: (category, datetime)
daily = (
    df.groupby("category").resample("D").agg(weighted_sum=("weighted_contrib", "sum"), total_weight=("weight", "sum"))
)

# 4. Pivot so each category is its own column
ws = daily["weighted_sum"].unstack("category").fillna(0)
tw = daily["total_weight"].unstack("category").fillna(0)

# 5. Compute N‑day rolling sums
ws_roll = ws.rolling(window=N, min_periods=1).sum()
tw_roll = tw.rolling(window=N, min_periods=1).sum()

# 6. Divide to get the rolling weighted avg
wavg_roll = ws_roll / tw_roll

# 7. Plot
fig = px.line(
    wavg_roll,
    title=f"{N}-Day Rolling Weighted‑Avg Effective Gas Price by Category",
    labels={"value": "Weighted‑Avg Effective Gas Price", "datetime": "Date"},
)
fig.update_yaxes(title_text="Effective Gas Price (Gwei)")
fig.show()

In [17]:
df.drop_duplicates()

,label,category,effective_gas_price,gas_used,gas_cost_in_eth,gas_price_times_gas_used,weighted_contrib,weight
datetime,,,,,,,,
2024-08-30 16:18:35+00:00,Custom Oracle Executor (pricing),pricing,4318952717,57192,0.000247,2.470095e+05,2.470095e+05,57192
2024-08-30 18:33:35+00:00,Custom Oracle Executor (pricing),pricing,4012396541,40629,0.000163,1.630197e+05,1.630197e+05,40629
2024-08-30 19:52:35+00:00,Custom Oracle Executor (pricing),pricing,2629695373,40629,0.000107,1.068419e+05,1.068419e+05,40629
2024-08-30 20:39:35+00:00,Custom Oracle Executor (pricing),pricing,2555462516,40653,0.000104,1.038872e+05,1.038872e+05,40653
2024-08-31 00:57:35+00:00,Custom Oracle Executor (pricing),pricing,2224735824,40653,0.000090,9.044219e+04,9.044219e+04,40653
...,...,...,...,...,...,...,...,...
2025-07-15 14:00:47+00:00,Calculators (keeper_network),keeper_network,6384563106,652353,0.004998,4.164989e+06,4.164989e+06,652353
2025-07-15 14:19:35+00:00,Eth Per Token Sender (keeper_network),keeper_network,4898790007,333030,0.001958,1.631444e+06,1.631444e+06,333030
2025-07-15 14:55:59+00:00,Incentive Pricing (keeper_network),keeper_network,15744489474,755116,0.014267,1.188892e+07,1.188892e+07,755116
